In [1]:
import pandas as pd
from conllu import parse

# Ввод

In [2]:
name = 'pisat_Dat'

object_form = {'Case': 'Dat', 'Animacy': 'Anim'}
verb = 'писать'
preposition = 'к'
if_preposition = False

# Данные и основные переменные

In [3]:
def create_path_and_file_name(name):
    path = '../data/{}.xlsx'.format(name)
    text_file_name = 'sents_{}.txt'.format(name)
    conllu_file_name = 'parsed_sents_{}.conllu'.format(name)
    file_name = [path, text_file_name, conllu_file_name]
    return file_name

In [4]:
names = create_path_and_file_name(name)

In [5]:
def read_preprocess_data(names):
    dataframe = pd.read_excel(names[0])
    dataframe.drop(columns=['Reversed left context', 'Reversed center', 'Title', 'Author', 'Birthday', 'Header', 
                            'Publ_year', 'Medium', 'Ambiguity', 'Publication', 'Sphere', 'Type', 'Topic'], inplace = True)
    return dataframe

In [6]:
df = read_preprocess_data(names)

In [7]:
def create_text_file(dataframe, names):
    with open(names[1], 'a', encoding = "UTF-8") as f:
        for i in range(dataframe.shape[0]):
            sent = dataframe['Full context'][i]
            f.write(sent[:sent.find('[')-1] + '\n')

In [8]:
def read_conllu(names):
    with open(names[2], 'r', encoding = "UTF-8") as f:
        parsed_sents = f.read()
        sents = parse(parsed_sents)
        return sents

In [9]:
create_text_file(df, names)

## Ввести названия файлов для парсера

In [10]:
print(names[1], '\n', names[2], sep = '')

sents_pisat_Dat.txt
parsed_sents_pisat_Dat.conllu


In [11]:
!udpipe --tokenize --tag --parse\
    ../models/russian-syntagrus-ud-2.0-170801.udpipe sents_pisat_Dat.txt > parsed_sents_pisat_Dat.conllu

Loading UDPipe model: done.


In [12]:
sents = read_conllu(names)

In [27]:
sents[0][15]

OrderedDict([('id', 16),
             ('form', 'в'),
             ('lemma', 'в'),
             ('upostag', 'ADP'),
             ('xpostag', None),
             ('feats', None),
             ('head', 18),
             ('deprel', 'case'),
             ('deps', None),
             ('misc', None)])

# Нахождение триплетов

In [47]:
def check_object(token, object_form):
    if 'Foreign' in token['feats'] and token['feats']['Foreign'] == 'Yes':
        return False
    for feature in object_form:
        if token['feats'][feature] != object_form[feature]:
            return False
    return True

In [48]:
def find_triplet_in_sentence(sent, verb, object_form, preposition=None, if_preposition=None):
    # TODO: подумать про случаи, когда несколько подходящих глаголов в предложении
    triplet = {}
    verb_id = None
    object_id = None
    for token in sent:
        
        # ищем id глагола, чтобы потом искать его зависимые
        if token['lemma'] == verb:
            verb_id = token['id']
            triplet['verb'] = token['form']
            
        # ищем зависимые (NB: в нашем случае зависимые всегда идут после глагола, поэтому в одном цикле)
        if token['head'] == verb_id:
            if token['upostag'] in ['NOUN', 'PROPN'] and check_object(token, object_form):
                object_id = token['id']
                break # мы нашли объект (TODO: bla bla)
                
    if preposition and object_id: # looking for a preposition, if there is
        if if_preposition:
            for token in sent:
                if token['head'] == object_id and token['form'] == preposition:
                    triplet['object'] = sent[object_id]['form']
                    triplet['preposition'] = preposition
        else:
            for token in sent: # looking for a preposition, if there isn't
                if token['head'] == object_id and token['form'] == preposition:
                    break
            else:
                triplet['object'] = sent[object_id]['form']
    return triplet

In [29]:
n = 0
print(sents[n])
find_triplet_in_sentence(sents[n], verb, object_form, preposition, if_preposition)

TokenList<Александру, Волошину, пришлось, писать, патриарху, Алексию, и, просить, обеспечить, невмешательство, его, ", подчинённых, ", (, в, светской, терминологии, ), в, избирательный, процесс, .>


{'verb': 'писать', 'object': 'Алексию'}

In [53]:
def get_all_triples(sentences, verb, object_form, preposition=None, if_preposition=None):
    triples = []
    for i, sent in enumerate(sentences):
        triplet = find_triplet_in_sentence(sent, verb, object_form, preposition, if_preposition)
        triplet['id'] = i
        triples.append(triplet)
    return triples

In [54]:
triples = get_all_triples(sents, verb, object_form, preposition, if_preposition)

In [55]:
triples

[{'verb': 'писать', 'object': 'Алексию', 'id': 0},
 {'verb': 'писал', 'id': 1},
 {'verb': 'написать', 'id': 2},
 {'verb': 'пишут', 'object': 'председатель', 'id': 3},
 {'verb': 'пишет', 'object': 'Госдумы', 'id': 4},
 {'verb': 'писала', 'object': '.', 'id': 5},
 {'verb': 'писал', 'object': 'компаний', 'id': 6},
 {'verb': 'пишет', 'object': 'Ленинградского', 'id': 7},
 {'id': 8},
 {'verb': 'писать', 'object': ',', 'id': 9},
 {'verb': 'писал', 'object': 'из', 'id': 10},
 {'verb': 'писал', 'object': 'Матвеевне', 'id': 11},
 {'verb': 'писала', 'object': 'Матвеевне', 'id': 12},
 {'verb': 'писал', 'object': ':', 'id': 13},
 {'verb': 'писала', 'id': 14},
 {'verb': 'писала', 'id': 15},
 {'verb': 'пишут', 'object': ',', 'id': 16},
 {'verb': 'пишешь', 'id': 17},
 {'verb': 'писал', 'id': 18},
 {'verb': 'писал', 'id': 19},
 {'id': 20},
 {'id': 21},
 {'id': 22},
 {'verb': 'писать', 'id': 23},
 {'verb': 'пишет', 'object': '!', 'id': 24},
 {'verb': 'Писали', 'object': ',', 'id': 25},
 {'verb': 'писал

# См. ниже: почему Госдумы считывается - это же совсем  другой падеж !!!

In [32]:
triples[0:45]


[{'verb': 'писать', 'object': 'Алексию'},
 {'verb': 'писал'},
 {'verb': 'написать'},
 {'verb': 'пишут'},
 {'verb': 'пишет', 'object': 'Госдумы'},
 {'verb': 'писала', 'object': '.'},
 {'verb': 'писал', 'object': 'компаний'},
 {'verb': 'пишет', 'object': 'Ленинградского'},
 {},
 {'verb': 'писать', 'object': ','},
 {'verb': 'писал'},
 {'verb': 'писал', 'object': 'Матвеевне'},
 {'verb': 'писала', 'object': 'Матвеевне'},
 {'verb': 'писал', 'object': ':'},
 {'verb': 'писала'},
 {'verb': 'писала'},
 {'verb': 'пишут', 'object': ','},
 {'verb': 'пишешь'},
 {'verb': 'писал'},
 {'verb': 'писал'},
 {},
 {},
 {},
 {'verb': 'писать'},
 {'verb': 'пишет', 'object': '!'},
 {'verb': 'Писали', 'object': ','},
 {'verb': 'писал'},
 {'verb': 'писал', 'object': '…'},
 {'verb': 'писала'},
 {'verb': 'писать', 'object': '…'},
 {'verb': 'писала', 'object': ','},
 {'verb': 'Пишет'},
 {'verb': 'писать', 'object': 'Дальстроя'},
 {'verb': 'пишут'},
 {'verb': 'писал', 'object': ','},
 {'verb': 'писать', 'object': ','

In [35]:
def count_triples(triples):
    count = 0
    for index, triple in enumerate(triples):
        if triple.get('object') != None:
            count+=1
#             dataframe.loc[index, {triple}] = triple['verb']
    return count

In [162]:
count_triples(triples)

270

## Пример
касаться до gen vs. касаться gen (от касаться зависит NOUN/PROPN + case-Gen (?) и не зависит до)